In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/home/kristbaum/Projects/reflectai/fatcat/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS available")
else:
    device = torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained("ibm/knowgl-large")
model = AutoModelForSeq2SeqLM.from_pretrained("ibm/knowgl-large").to(device)

GPU available
[(maidservant#Maidservant#profession)|field of this occupation|(milk#Milk#food)]$[(milk#Milk#food)|practiced by|(maidservant#Maidservant#profession)]


In [3]:
input_text = """The painting shows the Last Judgement, with God the Father at the top centre and Jesus just below him. 
To Jesus' right is his mother the Virgin Mary and to his left is Moses, holding the tablets showing the Ten Commandments. 
Rising up the left-hand side of the painting (at Jesus' right hand) are the blessed, whilst the damned fall into hell on the right-hand side. 
At the base of the painting are the dead, dying and those just resurrected. """

{'subject': {'name': 'maidservant', 'category': 'profession'}, 'verb': {'name': 'field of this occupation'}, 'object': {'name': 'milk', 'category': 'food'}}


In [4]:
import requests
import json

for key, value in parsed_statement.items():
    if "name" in value:
        if key == "subject" or key == "object":
            query = value["name"]
            print("Querying Wikidata for: " + query)
            url = f"https://wikidata.reconci.link/en/api?queries=%7B%22q0%22%3A%7B%22query%22%3A%22{query}%22%2C%22limit%22%3A1%7D%7D"
            response = requests.get(url)
            data = json.loads(response.text)
            value["result"] = {
                "description": data["q0"]["result"][0]["description"],
                "id": data["q0"]["result"][0]["id"],
                "wd_name": data["q0"]["result"][0]["name"],
            }
        elif key == "verb":
            query = value["name"]
            print("Querying Wikidata for: " + query)
            url = f"https://wikidata.reconci.link/en/suggest/property?prefix={query}"
            response = requests.get(url)
            data = json.loads(response.text)
            if data["result"][0]:
                value["result"] = (data["result"][0],)
            else:
                value["result"] = "No matching property found."


print(parsed_statement)

Querying Wikidata for: maidservant
Querying Wikidata for: field of this occupation
Querying Wikidata for: milk
{'subject': {'name': 'maidservant', 'category': 'profession', 'result': {'description': "female employee household work in the employer's home", 'id': 'Q833860', 'wd_name': 'maid'}}, 'verb': {'name': 'field of this occupation', 'result': ({'description': 'field corresponding to this occupation or profession (use only for occupations/professions - for people use Property:P101, for companies use P425)', 'id': 'P425', 'name': 'field of this occupation'},)}, 'object': {'name': 'milk', 'category': 'food', 'result': {'description': 'white liquid produced by the mammary glands of mammals', 'id': 'Q8495', 'wd_name': 'milk'}}}


In [5]:
print(json.dumps(parsed_statement, indent=4))


{
    "subject": {
        "name": "maidservant",
        "category": "profession",
        "result": {
            "description": "female employee household work in the employer's home",
            "id": "Q833860",
            "wd_name": "maid"
        }
    },
    "verb": {
        "name": "field of this occupation",
        "result": [
            {
                "description": "field corresponding to this occupation or profession (use only for occupations/professions - for people use Property:P101, for companies use P425)",
                "id": "P425",
                "name": "field of this occupation"
            }
        ]
    },
    "object": {
        "name": "milk",
        "category": "food",
        "result": {
            "description": "white liquid produced by the mammary glands of mammals",
            "id": "Q8495",
            "wd_name": "milk"
        }
    }
}


In [7]:
from pyvis.network import Network

graph = Network(height="800px", width="100%", notebook=True)

graph.add_node(parsed_statement["subject"]["result"]["id"], label=parsed_statement["subject"]["result"]["wd_name"])
graph.add_node(parsed_statement["object"]["result"]["id"], label=parsed_statement["object"]["result"]["wd_name"])
graph.add_edge(parsed_statement["subject"]["result"]["id"], parsed_statement["object"]["result"]["id"], label=parsed_statement["verb"]["name"])

graph.show("knowledge_graph.html")


knowledge_graph.html
